# Imports & Setup

In [18]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR') 

exchange = Exchange()
exchange.connect()


def test():
    #print(exchange.get_last_price_book())
    instruments = exchange.get_instruments()
    positions = exchange.get_positions()
    positions_and_cash = exchange.get_positions_and_cash()
#        public_trade_ticks = exchange.get_public_trade_ticks()
    private_trade_ticks = exchange.poll_new_trade_ticks("NVDA")
     #   order_book = exchange.get_order_book()
    last_price_book = exchange.get_last_price_book("NVDA")
    bid_volumes = last_price_book.bids
    ask_volumes = last_price_book.asks
    
    trade_history = exchange.get_trade_history("NVDA")
    outstanding_orders = exchange.get_outstanding_orders("NVDA")
    print("Positions: ", positions)
    print("Positions and Cash: ", positions_and_cash)
    #    print("Public Trade Ticks: ", public_trade_ticks)
    print("Private Trade Ticks: ", private_trade_ticks)
      #  print("Order Book: ", order_book)
    print("Last Price Book: ", last_price_book)
    print("Trade History: ", trade_history)
    print("Outstanding Orders: ", outstanding_orders)
    print("Instruments: ", instruments)
    #while True:
    print("asks: ", ask_volumes)
    print("bids: ", bid_volumes)
        
     #   time.sleep(1.0)

         
 

test()

2023-09-16 14:33:46,693 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2023-09-16 14:33:46,758 [client    ] [Thread-20   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


Positions:  {'NVDA': 9, 'ING': -15, 'SAN': -3, 'PFE': 15, 'CSCO': 3}
Positions and Cash:  {'NVDA': {'volume': 9, 'cash': -724.8}, 'ING': {'volume': -15, 'cash': 827.4}, 'SAN': {'volume': -3, 'cash': 146.7}, 'PFE': {'volume': 15, 'cash': -511.5}, 'CSCO': {'volume': 3, 'cash': -167.4000000000001}}
Private Trade Ticks:  []
Last Price Book:  PriceBook(NVDA 2023-09-16 14:33:46.714835)
 #bids | price | #asks 
       |  77.9 |   25  
       |  74.7 |  500  
       |  74.4 |  1115 
       |  74.3 |   70  
  320  |  74.1 |       
   65  |  74.0 |       
  1105 |  73.6 |       
   12  |  54.1 |       
   12  |  53.0 |       
   12  |  52.5 |       
   14  |  51.9 |       
Trade History:  []
Outstanding Orders:  {}
Instruments:  {'CSCO': Instrument(instrument_id=CSCO, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=50.0000, relative_change=50.00%), instrument_type=InstrumentType.STOCK, instrument_group=CSCO, paused=False, expired=False), 'PFE': Instrument(instrument_id=PFE, tic

# Function definitions

In [4]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

# Main algorithm

In [13]:
print(exchange.get_positions()['NVDA'])

5


In [17]:
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

# things we edit depending on the sentiment and liquidity
QUOTED_VOLUME = 3
FIXED_MINIMUM_CREDIT = 0.07
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100


def spread(bid_price, ask_price):
    # Spread & Liquidity
    bid_ask_spread = best_ask_price - best_bid_price 
    spread_percentage = (bid_ask_spread / best_ask_price) * 100
    print(f'Spread Info: {bid_ask_spread:>6.2f}, {spread_percentage:>6.2f}')
    
    return bid_ask_spread,spread_percentage

# 
def first_layer_evaluation: 
    # Obtain best bid and ask prices from order book to determine mid price
    best_bid_price = instrument_order_book.bids[0].price
    best_ask_price = instrument_order_book.asks[0].price
    mid_price = (best_bid_price + best_ask_price) / 2.0 
        
    # Calculate our fair/theoretical price based on the market mid price and our current position
    theoretical_price = mid_price #- PRICE_RETREAT_PER_LOT * position

    # Calculate final bid and ask prices to insert
    bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, instrument.tick_size)
    ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, instrument.tick_size)
        
        
        #bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, instrument.tick_size)
        #ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, instrument.tick_size)
        
        
        #bid_price = theoretical_price - FIXED_MINIMUM_CREDIT
        #ask_price = theoretical_price + FIXED_MINIMUM_CREDIT
        
    # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
    max_volume_to_buy = POSITION_LIMIT - position
    max_volume_to_sell = POSITION_LIMIT + position

    bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
    ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)

    # Display information for tracking the algorithm's actions
    print(f'{instrument.instrument_id:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')
        
    # Insert new quotes
    insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume)


while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')
    
    for instrument in INSTRUMENTS.values():
        # Remove all existing (still) outstanding limit orders
        exchange.delete_orders(instrument.instrument_id)
    
        # Obtain order book and only skip this instrument if there are no bids or offers available at all on that instrument,
        # as we we want to use the mid price to determine our own quoted price
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue
    
        # Obtain own current position in instrument
        position = exchange.get_positions()[instrument.instrument_id]
        # first layer for immediate response 
        
        
        
        
        
    # Wait for a few seconds to refresh the quotes
    print(f'\nWaiting for 2 seconds.')
    time.sleep(0.05)
    
    # Clear the displayed information after waiting
    clear_output(wait=True)


-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-09-16 00:22:38.549476 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :  -57
  ING       :   91
  SAN       :   64
  PFE       :  -25
  CSCO      :    5

PnL: -1731.70

          (ourbid) mktbid :: mktask (ourask)
  CSCO -- ( 65.18)  65.20 ::  65.30 ( 65.32)


2023-09-16 00:22:38,844 [client    ] [Thread-15   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


   PFE -- ( 27.53)  27.50 ::  27.70 ( 27.67)


KjException: (remote):0: failed: remote exception: expected !mWaitingToBeDisconnected; A disconnect was requested for this connection. No further messages are allowed
stack: 7fa02a3e3cc0 7fa02a3e2540 7fa02a3666d6 7fa02a354612 7fa02a354749 7fa02a349c74 7fa02a2b2639 7fa02a32b673 7fa04c91fcc0 7fa04c88fe4e 7fa03f3f4cba 7fa03f3f6293 7fa04c87f0a2 7fa04c91be68 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c926b01 7fa04c87f31d 7fa04c87f421 7fa04c87f0a2 7fa04c91be68 7fa04c925569